In [3]:
## --------------------- Retreive Information From Ebay Page ------------------
import re
from bs4 import BeautifulSoup
import requests

# ----------------------- Build URL -------------------------------------------

def get_page(url):
    response = requests.get(url)
    
    if not response.ok:
        print('Server responsed ', response.status_code)
    else:
        soup = BeautifulSoup(response.text, 'lxml')
    return soup

def format_url(word):
    starturl = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313.TR12.TRC2.A0.H0.Xgol.TRS0&_nkw='
    endurl = '&_sacat=0'
    words_split = word.split(" ")
    starturl = starturl + words_split[0]
    if len(words_split) == 1:
        pass
    else:
        for i in words_split[1:]:
            x = '%20' + i
            starturl = starturl + x
    finalurl = starturl + endurl
    return finalurl
      
def get_page_urls(soup):
    try:
        page2 = soup.find('a', class_ = 'x-pagination__control')
        more_links = page2.find_all('a', class_ = 's-item__link')
        links = soup.find_all('a', class_ = 's-item__link')
        moreurls = [item.get('href') for item in more_links]
    except:
        links = []
        moreurls = []
    
    urls = [item.get('href') for item in links]
    final = urls + moreurls 
    return urls

# ------------------------ Scrape Page ----------------------------------------
def get_price(soup):
    #retreives the price of current page item
    
    price = "0"
    try:
        price = soup.find('span', id = 'mm-saleDscPrc').text 
    except:
        pass
    try:
        price = soup.find('span', id = 'prcIsum').text
    except:
        pass
    
    price = re.sub("[^0-9.]", "", price)
    
    return float(price)

def get_rating_num(soup):
    #retreives the sellers rating of current page item
    
    rating = 0
    try:
        rating = soup.find('span', class_ = 'vi-mbgds3-bkImg  vi-mbgds3-fb5000-9999').text 
    except:
        pass
    try:
        rating = soup.find('span', class_ = 'mbg-l').find('a').text 
    except:
        pass
    
    return rating

def get_condition(soup):
    #retreives the condition of current page item
    con = ""
    try:
        con = soup.find('div', id = 'vi-itm-cond').text
    except:
        pass
    return con

def get_title(soup):
    #retreives the title of current page item
    try:
        title = soup.find('span', id = "vi-lkhdr-itmTitl").text
    except:
        title = ""
    return title

def get_positive_feedback(soup):
    #retreives the feedback rating of current page item
    feed = 0
    try:
        feed = soup.find('div', id = 'si-fb').text
        feed = float(feed.strip('%  Positive feedback').replace('%', ''))
    except:
        pass
    return feed

def get_sold(soup):
    #retreives the amount sold of current page item
    try:
        sold = int(soup.find('a', class_ = 'vi-txt-underline').text.strip('sold'))
    except:
        sold = 0
    return sold
        
def group_data(soup, url):
    #retreives every interested element of current page item and then
    #puts it all into a dictionary
    
    title = get_title(soup)
    sold = get_sold(soup)
    rating = get_rating_num(soup)
    price = get_price(soup)
    feedback = get_positive_feedback(soup)
    condition = get_condition(soup)
    url = url
    
    everything = {
        'title' : title,
        'price' : price,
        'sold' : sold,
        'rating': rating,
        'feedback' : feedback,
        'condition' : condition,
        'URL' : url
        }

    return everything





